In [2]:
import pandas as pd
import os

# Vérification des données
-> Format : vérifier format de toutes les colonnes

-> Valeurs manquante ( a faire -> déterminer les colonnes avec valeurs manquantes gestion des pb d'affichage)

-> Faire attention aux indexes

-> Choix des variables inutiles de tous garder
Pour l'instant variables utilisées (tourney_date, _id, _level, score, round, minutes, winner/loser_ rank, ioc, name, hand, w/l_bpSaved
 _bpFaced)


Rénommer fichier Travail_format_concat

## Fonctions de travail

In [17]:
var_interet = ["tourney_date", "tourney_id", "tourney_level", "score", "round", "minutes",
                "winner_rank", "winner_ioc", "winner_name", "winner_hand",
                "loser_rank", "loser_ioc", "loser_name", "loser_hand",
                 "w_bpSaved", "w_bpFaced", "l_bpSaved","l_bpFaced"]

In [3]:
# récupération de tous les matches simples (1968 à 2024)
def recup_match(type, genre, ind_deb, ind_fin):
    data_dict = {}
    formats_colonnes = {}

    if genre == 'atp':
        liste_fichier = os.listdir('../donnees_tennis/ATP')
    else:
        liste_fichier = os.listdir('../donnees_tennis/WTA')

    for fichier in liste_fichier:
        nom_deb = type + ind_deb
        nom_fin = type + ind_fin


        if fichier.startswith(nom_deb) or fichier.startswith(nom_fin):

            if genre =='atp':
                data = pd.read_csv('../donnees_tennis/ATP/' + fichier, index_col=False)
            else:
                data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)

            # Stockage du dataFrame
            data_dict[fichier] = data

            # Récupérer le format des colonnes pour ce DataFrame
            format_actuel = data_dict[fichier].dtypes.to_dict()
            formats_colonnes[fichier] = format_actuel

    return data_dict, formats_colonnes


In [6]:
def comp_format(formats_colonnes):

    print("\nComparaison des formats de colonnes entre les DataFrames:")
    print("------------------------------------------------------")

    # Vérifier si tous les DataFrames ont le même ensemble de colonnes
    noms_colonnes = None
    premiere_iteration = True
    for fichier, format_cols in formats_colonnes.items():
        colonnes_actuelles = set(format_cols.keys())
        if premiere_iteration:
            noms_colonnes = colonnes_actuelles
            premiere_iteration = False
        elif colonnes_actuelles != noms_colonnes:
            print(f"Erreur : Le fichier '{fichier}' a des colonnes différentes des autres DataFrames.")
            break
    else:
        # Si tous les DataFrames ont les mêmes colonnes, comparer les types
        premiere_iteration_types = True
        types_colonnes_reference = None
        for fichier, format_cols in formats_colonnes.items():
            if premiere_iteration_types:
                types_colonnes_reference = format_cols
                premiere_iteration_types = False
            elif format_cols != types_colonnes_reference:
                print(f"Erreur : Le fichier '{fichier}' a des formats de colonnes différents.")
                # Afficher les différences spécifiques pour ce fichier
                for colonne, type_ref in types_colonnes_reference.items():
                    if colonne in format_cols and format_cols[colonne] != type_ref:
                        print(f"  - La colonne '{colonne}' a le format '{format_cols[colonne]}' dans '{fichier}' au lieu de '{type_ref}'.")
                    elif colonne not in format_cols:
                        print(f"  - La colonne '{colonne}' est manquante dans '{fichier}'.")
                for colonne, type_act in format_cols.items():
                    if colonne not in types_colonnes_reference:
                        print(f"  - La colonne '{colonne}' est présente dans '{fichier}' mais pas dans le premier DataFrame.")
            else:
                print(f"Le fichier '{fichier}' a le même format de colonnes que les précédents.")

In [7]:
def modif_format(fichier, colonne, format, data_dict, formats_colonnes):
    data = data_dict[fichier]

    # On remplace les valeurs manquantes par 0 pour pouvoir changer le format
    data = data.fillna({colonne: 0})

    # Réassigner le résultat de astype à la colonne 'draw_size'
    data[colonne] = data[colonne].astype(format)


    data_dict[fichier] = data
    formats_colonnes[fichier] = data.dtypes.to_dict()

    return data_dict, formats_colonnes

In [23]:
def afficher(data):
    descriptif = pd.DataFrame({
        'Type': data.dtypes,
        'Nombre de valeurs manquantes': data.isnull().sum(),
        'Pourcentage de valeurs manquantes': (data.isnull().sum() / len(data)) * 100
    })
    display(descriptif)

In [5]:
def concatenation(data_dict):
    data_fin = pd.DataFrame()

    for data in data_dict.values():
        data_fin = pd.concat([data_fin, data], axis=0)

    return data_fin

## ATP

### matches simple

In [8]:
data_dict, formats_colonnes = recup_match("atp_matches","atp","_19","_20")

In [9]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'atp_matches_1969.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1971.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1972.csv' au lieu de 'float64'.
Erreur : Le fichier 'atp_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'int64' dans 'atp_matches_1973.csv' au lieu de 'float6

In [10]:
data_dict['atp_matches_1968.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32.0,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32.0,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32.0,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32.0,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32.0,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_dict, formats_colonnes = modif_format('atp_matches_1968.csv','draw_size', 'int64', data_dict, formats_colonnes)

In [12]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1978.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_1979.csv' a le même format de co

Concaténation

In [13]:
atp_matches_1968_2024 = concatenation(data_dict)

Selection des variables d'intérets

In [18]:
atp_matches_1968_2024 = atp_matches_1968_2024[var_interet]

In [24]:
afficher(atp_matches_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
tourney_date,int64,0,0.000000
tourney_id,object,0,0.000000
tourney_level,object,0,0.000000
score,object,9,0.004615
round,object,0,0.000000
minutes,float64,99891,51.227205
winner_rank,float64,35775,18.346530
winner_ioc,object,8,0.004103
winner_name,object,0,0.000000
winner_hand,object,3,0.001538


Sauvegarde

In [19]:

atp_matches_1968_2024.to_csv("../Donnees/atp_matches_1968_2024.csv")

### matchs futures

In [25]:
data_dict, formats_colonnes = recup_match("atp_matches_futures","atp","_19","_20")

In [26]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_futures_1992.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1993.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1994.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1995.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1996.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1997.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1998.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1999.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2000.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2001.csv' a le même format de colon

In [27]:
data_dict['atp_matches_futures_2020.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,101,144869,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,330.0,117.0,560.0,44.0
1,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,102,208262,NaN,WC,...,NaN,NaN,NaN,NaN,NaN,NaN,857.0,15.0,1670.0,1.0
2,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,103,105607,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,852.0,15.0,1892.0,1.0
3,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,104,106032,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,342.0,110.0,903.0,13.0
4,2020-M-ITF-ARG-01A-2020,M25 Rio Cuarto,Clay,32,25,20200224,105,207678,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,371.0,97.0,1212.0,5.0


In [28]:
data_dict['atp_matches_futures_2019.csv'].head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,1,126135,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,618.0,3.0,338.0,67.0
1,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,2,200553,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,415.0,27.0,403.0,30.0
2,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,3,145001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,466.0,16.0,503.0,11.0
3,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,4,106187,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,359.0,57.0,413.0,28.0
4,2019-M-ITF-HKG-01A-2019,M25 Hong Kong,Hard,32,S,20181231,5,134120,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354.0,58.0


On choisit de modifier la variables "tourney_level" en str

In [29]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,"tourney_level", 'str', data_dict, formats_colonnes)

In [30]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_futures_1992.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1993.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1994.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1995.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1996.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1997.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1998.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_1999.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2000.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_futures_2001.csv' a le même format de colon

Concaténation

In [32]:
atp_matches_futures_1992_2024 = concatenation(data_dict)

In [33]:
atp_matches_futures_1992_2024 = atp_matches_futures_1992_2024[var_interet]
afficher(atp_matches_futures_1992_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
tourney_date,int64,0,0.000000
tourney_id,object,0,0.000000
tourney_level,object,0,0.000000
score,object,33,0.006619
round,object,0,0.000000
minutes,float64,498555,100.000000
winner_rank,float64,39346,7.892008
winner_ioc,object,0,0.000000
winner_name,object,0,0.000000
winner_hand,object,0,0.000000


Sauvegarde

In [34]:
atp_matches_futures_1992_2024.to_csv("../Donnees/atp_matches_futures_1992_2024.csv")

### matchs qualificatifs

In [35]:
data_dict, formats_colonnes = recup_match("atp_matches_qual_chall","atp","_19","_20")

In [36]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_matches_qual_chall_1979.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1980.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1981.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1982.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1983.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1984.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1985.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1986.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1987.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_matches_qual_chall_1988.

Concaténation

In [38]:
atp_matches_qual_1978_2024 = concatenation(data_dict)

In [39]:
atp_matches_qual_1978_2024 = atp_matches_qual_1978_2024[var_interet]
afficher(atp_matches_qual_1978_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
tourney_date,int64,0,0.000000
tourney_id,object,0,0.000000
tourney_level,object,0,0.000000
score,object,77,0.034514
round,object,0,0.000000
minutes,float64,111056,49.779244
winner_rank,float64,8494,3.807313
winner_ioc,object,0,0.000000
winner_name,object,0,0.000000
winner_hand,object,0,0.000000


Sauvegarde

In [40]:
atp_matches_qual_1978_2024.to_csv("../Donnees/atp_matches_qual_1978_2024.csv")

### Rankings rangs

In [41]:
data_dict, formats_colonnes = recup_match("atp_rank","atp","","")

In [42]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_rankings_10s.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'atp_rankings_20s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'int64' dans 'atp_rankings_20s.csv' au lieu de 'float64'.
Le fichier 'atp_rankings_70s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_90s.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'atp_rankings_current.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'int64' dans 'atp_rankings_current.csv' au lieu de 'float64'.


In [43]:
data_dict['atp_rankings_10s.csv'].head()

,ranking_date,rank,player,points
0,20100104,1,103819,10550.0
1,20100104,2,104745,9205.0
2,20100104,3,104925,8310.0
3,20100104,4,104918,7030.0
4,20100104,5,105223,6785.0


In [44]:
data_dict['atp_rankings_20s.csv'].head()

,ranking_date,rank,player,points
0,20200106,1,104745,9985
1,20200106,2,104925,9055
2,20200106,3,103819,6590
3,20200106,4,106233,5825
4,20200106,5,106421,5705


In [45]:
data_dict, formats_colonnes = modif_format('atp_rankings_20s.csv','points', 'float64', data_dict, formats_colonnes)
data_dict, formats_colonnes = modif_format('atp_rankings_current.csv','points', 'float64', data_dict, formats_colonnes)

In [46]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'atp_rankings_10s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_20s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_70s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_90s.csv' a le même format de colonnes que les précédents.
Le fichier 'atp_rankings_current.csv' a le même format de colonnes que les précédents.


Concatenatiion

In [47]:
atp_rankings = concatenation(data_dict)

In [48]:
afficher(atp_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
ranking_date,int64,0,0.000000
rank,int64,0,0.000000
player,int64,0,0.000000
points,float64,376080,11.420766


In [102]:
atp_rankings = atp_rankings[["ranking_date", "rank", "player"]]
afficher(atp_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
ranking_date,int64,0,0.0
rank,int64,0,0.0
player,int64,0,0.0


Sauvegarde

In [103]:
atp_rankings.to_csv("../Donnees/atp_rankings.csv")

## WTA

### matches simple

In [50]:
data_dict, formats_colonnes = recup_match("wta_matches","wta","_19","_20")

In [52]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_matches_1969.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1971.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_

In [53]:
data_dict['wta_matches_1968.csv']["winner_entry"].unique()

array([nan])

In [54]:
data_dict['wta_matches_1969.csv']["winner_entry"].unique()

array([nan, 'LL', 'Q'], dtype=object)

In [55]:
data_dict, formats_colonnes = modif_format('wta_matches_1968.csv',"winner_entry", 'str', data_dict, formats_colonnes)

In [56]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'object' dans 'wta_matches_1970.csv' au lieu de 'float64'.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1973.csv' au

In [57]:
data_dict['wta_matches_1968.csv']["draw_size"].unique()

array([13., 11., 62., 96., 64.,  4., 32., 59., 16., 15., 31., 38., 23.,
        8., 55., 12., 29., 25., 27., 24., nan])

In [58]:
data_dict['wta_matches_1970.csv']["draw_size"].unique()

array(['43', '96', '64', '4', '16', '8', '12', '31', '32', '40', '27',
       '24', '48', '25', '28', '30', '6', '15', nan, 'exho'], dtype=object)

Ajout du type exho pour exhibition -> modification de toute les valeur en str

In [59]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,"draw_size", 'str', data_dict, formats_colonnes)


In [60]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1972.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1973.csv' a des formats de colonnes différents.
  - La colonne 'winner_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
  - La colonne 'loser_seed' a le format 'float64' dans 'wta_matches_1973.csv' au lieu de 'object'.
Erreur : Le fichier 'wta_matches_1974.csv' a des formats de colonnes différen

In [61]:
data_dict['wta_matches_1971.csv']['winner_seed'].unique()

array([nan, '8', '5', '7', '1', '2', '3', '6', '4', '8F', '3F', '5F',
       '2F', '6F', '4F', '7F', '1F', '12', '14', '9', '10', '13', '15',
       '11'], dtype=object)

In [62]:
data_dict['wta_matches_1972.csv']['winner_seed'].unique()

array([ 1., nan,  8.,  5.,  4.,  3.,  6.,  7.,  2., 16.,  9., 12., 13.,
       14., 10., 15.])

In [63]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,'winner_seed', 'str', data_dict, formats_colonnes)
    data_dict, formats_colonnes = modif_format(fichier,'loser_seed', 'str', data_dict, formats_colonnes)

In [64]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1978.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_1979.csv' a le même format de co

Concaténation

In [66]:
wta_matches_1968_2024 = concatenation(data_dict)

In [67]:
wta_matches_1968_2024 = wta_matches_1968_2024[var_interet]
afficher(wta_matches_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
tourney_date,int64,0,0.000000
tourney_id,object,0,0.000000
tourney_level,object,0,0.000000
score,object,1,0.000633
round,object,0,0.000000
minutes,float64,135965,86.003719
winner_rank,float64,51616,32.649343
winner_ioc,object,30,0.018976
winner_name,object,0,0.000000
winner_hand,object,0,0.000000


Enregistrement

In [68]:
wta_matches_1968_2024.to_csv("../Donnees/wta_matches_1968_2024.csv")

### matchs qualificatifs

In [69]:
data_dict, formats_colonnes = recup_match("wta_matches_qual","wta","","")

C:\Users\33663\AppData\Local\Temp\ipykernel_92432\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_92432\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_92432\2525029270.py:21: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)
C:\Users\33663\AppData\Local\Temp\ipykernel_92432\2525029270.py:21: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../donnees_tennis/WTA/' + fichier, index_col=False)


In [70]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_matches_qual_itf_1969.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1969.csv' au lieu de 'int64'.
  - La colonne 'winner_seed' a le format 'object' dans 'wta_matches_qual_itf_1969.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1970.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1970.csv' au lieu de 'int64'.
  - La colonne 'winner_seed' a le format 'object' dans 'wta_matches_qual_itf_1970.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1971.csv' a des formats de colonnes différents.
  - La colonne 'draw_size' a le format 'float64' dans 'wta_matches_qual_itf_1971.csv' au lieu de 'int64'.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_197

In [71]:
data_dict['wta_matches_qual_itf_1968.csv']['draw_size'].unique()

array([32, 16, 15, 31, 29, 96], dtype=int64)

In [72]:
data_dict['wta_matches_qual_itf_1968.csv']['winner_seed'].unique()

array([ 5., nan,  6.,  4.])

In [73]:
data_dict['wta_matches_qual_itf_1969.csv']['draw_size'].unique()

array([16., 32., 31., 30., nan, 64., 96.])

In [74]:
data_dict['wta_matches_qual_itf_1969.csv']['winner_seed'].unique()

array([nan, '4', '6', '5', '3', '7', '8', '2F'], dtype=object)

In [75]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,'winner_seed', 'str', data_dict, formats_colonnes)
    data_dict, formats_colonnes = modif_format(fichier,'draw_size', 'int64', data_dict, formats_colonnes)

In [76]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Erreur : Le fichier 'wta_matches_qual_itf_1971.csv' a des formats de colonnes différents.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_1971.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_matches_qual_itf_1972.csv' a des formats de colonnes différents.
  - La colonne 'winner_entry' a le format 'object' dans 'wta_matches_qual_itf_1972.csv' au lieu de 'float64'.
  - La colonne 'loser_entry' a le format 'object' dans 'wta_matches_qual_itf_1972.csv' au lieu de 'float64'.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Erreur : L

In [77]:
data_dict['wta_matches_qual_itf_1968.csv']['loser_entry'].unique()

array([nan])

In [78]:
data_dict['wta_matches_qual_itf_1968.csv']['winner_entry'].unique()

array([nan])

In [79]:
data_dict['wta_matches_qual_itf_1972.csv']['loser_entry'].unique()

array([nan, 'Q'], dtype=object)

In [80]:
data_dict['wta_matches_qual_itf_1972.csv']['winner_entry'].unique()

array([nan, 'Q'], dtype=object)

In [81]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,'winner_entry', 'str', data_dict, formats_colonnes)
    data_dict, formats_colonnes = modif_format(fichier,'loser_entry', 'str', data_dict, formats_colonnes)

In [82]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1978.csv' a le même forma

In [83]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,'loser_seed', 'str', data_dict, formats_colonnes)

In [84]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_matches_qual_itf_1969.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1970.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1971.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1972.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1973.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1974.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1975.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1976.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1977.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_matches_qual_itf_1978.csv' a le même forma

Concatenation

In [86]:
wta_matches_qual_1968_2024 = concatenation(data_dict)

In [87]:
wta_matches_qual_1968_2024 = wta_matches_qual_1968_2024[var_interet]
afficher(wta_matches_qual_1968_2024)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
tourney_date,int64,0,0.000000
tourney_id,object,0,0.000000
tourney_level,object,0,0.000000
score,object,196,0.033003
round,object,0,0.000000
minutes,float64,576868,97.134630
winner_rank,float64,124575,20.976283
winner_ioc,object,40,0.006735
winner_name,object,0,0.000000
winner_hand,object,101,0.017007


Enregistrement

In [88]:
wta_matches_qual_1968_2024.to_csv("../Donnees/wta_matches_qual_1968_2024.csv")

### rankings

In [89]:
data_dict, formats_colonnes = recup_match("wta_rank","wta","","")

In [90]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Erreur : Le fichier 'wta_rankings_10s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_10s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_20s.csv' a des formats de colonnes différents.
  - La colonne 'tours' a le format 'int64' dans 'wta_rankings_20s.csv' au lieu de 'float64'.
Erreur : Le fichier 'wta_rankings_80s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_80s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_90s.csv' a des formats de colonnes différents.
  - La colonne 'points' a le format 'float64' dans 'wta_rankings_90s.csv' au lieu de 'int64'.
Erreur : Le fichier 'wta_rankings_current.csv' a des formats de colonnes différents.
  - La colonne 'tours' a le format 'int64' dans 'wta_rankings_current.csv' au lieu de 'float64'.


In [91]:
for fichier in data_dict.keys():
    data_dict, formats_colonnes = modif_format(fichier,'points', 'int64', data_dict, formats_colonnes)
    data_dict, formats_colonnes = modif_format(fichier,'tours', 'int64', data_dict, formats_colonnes)


In [92]:
comp_format(formats_colonnes)


Comparaison des formats de colonnes entre les DataFrames:
------------------------------------------------------
Le fichier 'wta_rankings_10s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_20s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_80s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_90s.csv' a le même format de colonnes que les précédents.
Le fichier 'wta_rankings_current.csv' a le même format de colonnes que les précédents.


Concaténation

In [94]:
wta_rankings = concatenation(data_dict)

In [95]:
afficher(wta_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
ranking_date,int64,0,0.0
rank,int64,0,0.0
player,int64,0,0.0
points,int64,0,0.0
tours,int64,0,0.0


In [99]:
wta_rankings = wta_rankings[["ranking_date", "rank", "player"]]
afficher(wta_rankings)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
ranking_date,int64,0,0.0
rank,int64,0,0.0
player,int64,0,0.0


Enregistrement

In [100]:
wta_rankings.to_csv("../Donnees/wta_rankings.csv")